In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly import graph_objects as go
import json
import ast
import re

In [3]:
# Configure pandas to show all columns without truncation
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows
pd.set_option('display.width', None)        # Don't wrap output
pd.set_option('display.max_colwidth', None) # Show full content in each column

In [6]:
# Get all column names
all_cols = df.columns.tolist()

# Metadata columns (not features)
metadata_cols = ['id', 'shot_id', 'participant_id']

# Target columns (what you're predicting)
target_cols = ['angle', 'depth', 'left_right']

# Input features (everything else)
input_features = [col for col in all_cols if col not in metadata_cols + target_cols]

print("="*60)
print(f"TOTAL COLUMNS: {len(all_cols)}")
print("="*60)

print(f"\nMETADATA ({len(metadata_cols)} columns):")
print(metadata_cols)

print(f"\nINPUT FEATURES ({len(input_features)} columns):")
for i, feat in enumerate(input_features, 1):
    print(f"{i:3d}. {feat}")

print(f"\nTARGET VARIABLES ({len(target_cols)} columns):")
for i, target in enumerate(target_cols, 1):
    print(f"{i}. {target}")

print("\n" + "="*60)
print(f"Summary: {len(input_features)} features → {len(target_cols)} targets")
print("="*60)

TOTAL COLUMNS: 213

METADATA (3 columns):
['id', 'shot_id', 'participant_id']

INPUT FEATURES (207 columns):
  1. nose_x
  2. nose_y
  3. nose_z
  4. left_eye_x
  5. left_eye_y
  6. left_eye_z
  7. right_eye_x
  8. right_eye_y
  9. right_eye_z
 10. left_ear_x
 11. left_ear_y
 12. left_ear_z
 13. right_ear_x
 14. right_ear_y
 15. right_ear_z
 16. left_shoulder_x
 17. left_shoulder_y
 18. left_shoulder_z
 19. right_shoulder_x
 20. right_shoulder_y
 21. right_shoulder_z
 22. left_elbow_x
 23. left_elbow_y
 24. left_elbow_z
 25. right_elbow_x
 26. right_elbow_y
 27. right_elbow_z
 28. left_wrist_x
 29. left_wrist_y
 30. left_wrist_z
 31. right_wrist_x
 32. right_wrist_y
 33. right_wrist_z
 34. left_hip_x
 35. left_hip_y
 36. left_hip_z
 37. right_hip_x
 38. right_hip_y
 39. right_hip_z
 40. left_knee_x
 41. left_knee_y
 42. left_knee_z
 43. right_knee_x
 44. right_knee_y
 45. right_knee_z
 46. left_ankle_x
 47. left_ankle_y
 48. left_ankle_z
 49. right_ankle_x
 50. right_ankle_y
 51. right

In [5]:
df = pd.read_csv('train.csv')
df.head(1)

id  shot_id  participant_id  \
0  4438ed82-c13d-48d3-b1eb-fa39fb2832c6  qxapevM               1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [7]:
# Extract unique body parts being measured (removing _x, _y, _z suffixes)
body_parts = set()
for col in input_features:
    # Remove _x, _y, _z suffix to get the body part name
    if col.endswith('_x') or col.endswith('_y') or col.endswith('_z'):
        body_part = col.rsplit('_', 1)[0]  # Remove last underscore and suffix
        body_parts.add(body_part)

body_parts = sorted(body_parts)

print("="*60)
print("TARGET VARIABLES (what we're predicting)")
print("="*60)
print("\n1. angle      - Shot angle (degrees)")
print("2. depth      - Shot depth/distance")
print("3. left_right - Horizontal position")

print("\n" + "="*60)
print(f"BODY PARTS BEING TRACKED ({len(body_parts)} parts)")
print("="*60)
print("\nEach body part has X, Y, Z coordinates tracked over time:\n")

# Group by body region for better readability
face_parts = [p for p in body_parts if any(x in p for x in ['nose', 'eye', 'ear'])]
upper_body = [p for p in body_parts if any(x in p for x in ['shoulder', 'elbow', 'wrist', 'neck'])]
lower_body = [p for p in body_parts if any(x in p for x in ['hip', 'knee', 'ankle', 'toe', 'heel'])]
hand_parts = [p for p in body_parts if any(x in p for x in ['finger', 'thumb', 'pinky']) and p not in upper_body]

print("FACE:")
for i, part in enumerate(face_parts, 1):
    print(f"  {i:2d}. {part}")

print("\nUPPER BODY:")
for i, part in enumerate(upper_body, 1):
    print(f"  {i:2d}. {part}")

print("\nLOWER BODY:")
for i, part in enumerate(lower_body, 1):
    print(f"  {i:2d}. {part}")

print("\nHANDS (detailed finger tracking):")
for i, part in enumerate(hand_parts, 1):
    print(f"  {i:2d}. {part}")

print("\n" + "="*60)
print(f"TOTAL: {len(body_parts)} body parts × 3 axes = {len(input_features)} features")
print("="*60)

TARGET VARIABLES (what we're predicting)

1. angle      - Shot angle (degrees)
2. depth      - Shot depth/distance
3. left_right - Horizontal position

BODY PARTS BEING TRACKED (69 parts)

Each body part has X, Y, Z coordinates tracked over time:

FACE:
   1. left_ear
   2. left_eye
   3. nose
   4. right_ear
   5. right_eye

UPPER BODY:
   1. left_elbow
   2. left_shoulder
   3. left_wrist
   4. neck
   5. right_elbow
   6. right_shoulder
   7. right_wrist

LOWER BODY:
   1. left_ankle
   2. left_big_toe
   3. left_heel
   4. left_hip
   5. left_knee
   6. left_small_toe
   7. mid_hip
   8. right_ankle
   9. right_big_toe
  10. right_heel
  11. right_hip
  12. right_knee
  13. right_small_toe

HANDS (detailed finger tracking):
   1. left_fifth_finger_dip
   2. left_fifth_finger_distal
   3. left_fifth_finger_mcp
   4. left_fifth_finger_pip
   5. left_first_finger_cmc
   6. left_first_finger_distal
   7. left_first_finger_ip
   8. left_first_finger_mcp
   9. left_fourth_finger_dip
  10